# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [14]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   NULL|     BE|   NULL|    NULL|      1|  NULL|   269|  6|    69| NULL|       1|    NULL|    0.0|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070802| 1963| 1096|   NULL|     US|     TX|    NULL|      1|  NULL|     2|  6|    63| NULL|       0|    NULL|   NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070803| 1963| 1096|   NULL|     US|     IL|    NULL|      1|  NULL|     2|  6|    6

In [17]:
cited = patents.alias("cited")
citing = patents.alias("citing")

In [19]:
midway = citations.join(citing, citing.PATENT == citations.CITING)
midway.show(5)

+-------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| CITING|  CITED| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3858242|1515701|3858242| 1975| 5485|   1973|     US|     MI|    NULL|      1|     5|     2|  6|    63|    4|      12|    0.75| 0.1528|     0.0| 11.0833|   16.25|    NULL|    NULL|    NULL|    NULL|
|3858242|3319261|3858242| 1975| 5485|   1973|     US|     MI|    NULL|      1|     5|     2|  6|    63|    4|      12|    0.75| 0.1528|     0.0| 11.0833|   16.25|    NULL|    NULL|    NULL|    NULL|
|3858

In [26]:
fully_joined = midway.join(cited, cited.PATENT == citations.CITED)

+-------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| CITING|  CITED| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-----+-----+-------+-------+------

In [27]:
fully_joined = fully_joined.select("CITING", "CITED", "citing.POSTATE", "cited.POSTATE")
fully_joined.show(5)

+-------+-------+-------+-------+
| CITING|  CITED|POSTATE|POSTATE|
+-------+-------+-------+-------+
|4483021|3070803|     MS|     IL|
|4133055|3070803|     NH|     IL|
|4253313|3070803|   NULL|     IL|
|5054122|3070803|   NULL|     IL|
|5557807|3070803|     FL|     IL|
+-------+-------+-------+-------+
only showing top 5 rows



In [30]:
non_null = fully_joined.filter(fully_joined["cited.POSTATE"] != "NONE")
non_null = non_null.filter(non_null["citing.POSTATE"] != "NONE")
non_null.show(5)

+-------+-------+-------+-------+
| CITING|  CITED|POSTATE|POSTATE|
+-------+-------+-------+-------+
|4483021|3070803|     MS|     IL|
|4133055|3070803|     NH|     IL|
|4921141|3070803|     CA|     IL|
|5557807|3070803|     FL|     IL|
|4484363|3070803|     CA|     IL|
+-------+-------+-------+-------+
only showing top 5 rows



In [31]:
same_state = non_null.filter(non_null["citing.POSTATE"] == non_null["cited.POSTATE"])
same_state.show(5)

+-------+-------+-------+-------+
| CITING|  CITED|POSTATE|POSTATE|
+-------+-------+-------+-------+
|4067198|3217791|     AK|     AK|
|4676695|3217791|     AK|     AK|
|5190098|3217791|     AK|     AK|
|5238053|3217791|     AK|     AK|
|4075779|3373523|     AK|     AK|
+-------+-------+-------+-------+
only showing top 5 rows



In [32]:
counted = same_state.groupBy("CITING").count()
counted.show(5)

+-------+-----+
| CITING|count|
+-------+-----+
|3968160|    1|
|5292488|    3|
|5179246|    1|
|5392941|    3|
|5734569|    2|
+-------+-----+
only showing top 5 rows



In [35]:
ordered = counted.sort(counted["count"].desc())
ordered.show(10)

+-------+-----+
| CITING|count|
+-------+-----+
|5959466|  125|
|5983822|  103|
|6008204|  100|
|5952345|   98|
|5998655|   96|
|5958954|   96|
|5936426|   94|
|5951547|   90|
|5978329|   90|
|5739256|   90|
+-------+-----+
only showing top 10 rows

